## 특정 키워드로 유튜브 검색 후 영상제목 및 조회수 크롤링
- 썸네일 크기가 작은 영상들은 제외하고 메인 영상들만 수집하기

In [3]:
from selenium import webdriver as wb
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import time
import pandas as pd
from tqdm import tqdm

In [4]:
driver = wb.Chrome()
driver.maximize_window()
keyword = input("검색 키워드 입력 :")
driver.get(f"https://www.youtube.com/results?search_query={keyword}")

검색 키워드 입력 : 뉴스


### 메인 영상 제목 가져오기
- 가로로 움직이는 썸네일 크기가 작은 영상들(주요뉴스, 작은 쇼츠) 제외

In [5]:
titles = driver.find_elements(By.CSS_SELECTOR, '#video-title > yt-formatted-string')
print("제목 개수 : ", len(titles))
for t in titles:
    print(t.text)

제목 개수 :  20
[뉴스 '꾹'] "뭐야 저거!" 청와대에 '사우디' 시뻘건 낙서한 여성 태연히.. (2025.07.19/MBC뉴스)
[🔴속보] '조사 거부' 尹 바로 기소…"또 그러면" 특검의 무서운 경고｜"실효성 있는 조사 담보 어려워…尹 행태 양형 반영" / 연합뉴스TV(YonhapnewsTV)
'마산 다 망하게 생겼습니다'.. 李정부 인사 향해 '우르르' [뉴스.zip/MBC뉴스]
경기 북부·강원 호우특보..이 시각 기상센터 - [LIVE] MBC 뉴스특보 2025년 07월 20일
극동방송·순복음 '우리도?' 전격 압수수색에 발칵 [뉴스.zip/MBC뉴스]
경기 북부에 호우 특보…제보로 보는 피해 상황 [뉴스START] 다시보기 2025년 07월 20일 / YTN
[LIVE][KBS 뉴스특보] 산청 집중호우로 8명 사망·2명 중상·6명 실종…구조 58명/ 마을 삼킨 '살인 폭우' 산사태 비상...뉴스특보/2025년 7월 19일(토)/KBS
'나윤장송·쌍권 아니고 나?' 징계 통보서 '어이상실' [뉴스.zip/MBC뉴스]
여기 뉴욕 맞아? 지하철까지 물폭탄 '콸콸', 거대 구름장벽 쓰나미처럼..유럽 '펄펄'-[핫이슈PLAY] MBC뉴스 2025년 7월 20일
경기 가평 폭우로 사망·실종 4명‥"연락두절 8명" - [LIVE] MBC 뉴스특보 2025년 07월 20일
"맞다 이 사람 대선후보였지.." '정동영 답변' 화제된 이유 [뉴스.zip/MBC뉴스]
"조작은 패가망신" 경고했는데..'빼박불가'? 딱 걸린 방시혁 [뉴스.zip/MBC뉴스]
[🔴속보] 집중 호우로 경기 가평서 2명 숨져…경찰, 비상 근무/2025년 7월 20일(일)/KBS
[🔴LIVE][KBS 뉴스특보] 중대본 "사망·실종 26명.. 이재민 1만3천여명 발생"/2025년 7월 20일(일)/KBS
'진짜 해산되나' 간단치 않다.. 與 당권주자 협공에 '벌컥' [뉴스.zip/MBC뉴스]
이 시각 집중호우-하천수위 상황 / '폭우 구름' 다시 남부 유입..경상권 최대 200mm 집중호우 우

### 조회수 가져오기
- '시청 중'도 같이 수집

In [9]:
views= driver.find_elements(By.CSS_SELECTOR, '#metadata-line > span:nth-child(3)')     # copy selector로가져오면 3번째임
print("조회수 개수 : ", len(views))
for v in views:
    print(v.text)

조회수 개수 :  20
조회수 68만회
조회수 9.6만회
조회수 75만회
조회수 24만회
조회수 21만회
조회수 7.8천회
조회수 24만회
조회수 13만회
2.8천명 시청 중
조회수 9.1만회
조회수 36만회
조회수 77만회
598명 시청 중
88명 시청 중
조회수 22만회
조회수 21만회
조회수 4.5만회
조회수 22만회
조회수 120만회
조회수 41만회


#### 유튜브 웹 페이지의 특성상 검색어나 검색 시점에 따라서 출력되는 영상의 포맷들이나 형태가 다 달라짐(변동성이 매우 큰 웹 페이지)
- 메인 영상과 주요 뉴스와 Short도 출력 형태가 다른데 영상 전체를 감싸는 태그의 내부 구조가 동일함
- 따라서 정확하게 원하는 부분에만 접근하기 위해서는 3 종류의 영상 차이가 발생하는 부분을 잘 찾아내는 것이 중요함
  - 메인영상 : ytd-video-renderer 태그
  - 주요뉴스 : ytd-grid-video-renderer 태그
  - Short : ytd-reel-item-renderer 태그

### while문으로 스크롤을 어느정도 내려서 좀 더 많은 수의 제목과 조회수를 뽑아보자

In [10]:
# 영상제목과 조회수 HTML 요소들이 담길 리스트

titles_list_html, views_list_html = [], []          # ctrl +shift + l = 한번에 같은 문자 수정

# 수집 영상의 개수를 50개 이상이 될때까지 반복
while len(titles_list_html) < 50 :          # 50개가 안됐으니 스크롤을 계속 내리
    body = driver.find_element(By.TAG_NAME, 'body')
    body.send_keys(Keys.END)
    time.sleep(3)      # 영상이라서 텍스트나 이미지 데이터보다 더 무거울 수 있으므로 조금 더 시간을 많이 두기

    titles_list_html = driver.find_elements(By.CSS_SELECTOR, '#video-title > yt-formatted-string')
    views_list_html = driver.find_elements(By.CSS_SELECTOR, '#metadata-line > span:nth-child(3)')

print("수집된 총 제목 개수 :" , len(titles_list_html))
print("수집된 총 조회수 개수 :" , len(views_list_html))

# 우리가 원하는 영상 제목과 조회수가 담길 빈 리스트
title_list, view_list = [] , []


for i in tqdm(range(len(titles_list_html))) :
    title_list.append(titles_list_html[i].text)
    view_list.append(views_list_html[i].text)

print("영상 제목 수 :", len(title_list))
print("영상 조회 수 :", len(view_list))

수집된 총 제목 개수 : 59
수집된 총 조회수 개수 : 59


100%|██████████████████████████████████████████████████████████████████████████████████| 59/59 [00:01<00:00, 54.99it/s]

영상 제목 수 : 59
영상 조회 수 : 59


### 정확히 영상제목과 조회수를 50개 지정하여 추출해보세요.

In [ ]:
titles_list_html, views_list_html = [], []       

while len(titles_list_html) < 50 :
    body = driver.find_element(By.TAG_NAME, 'body')
    body.send_keys(Keys.END)
    time.sleep(3)      

    titles_list_html = driver.find_elements(By.CSS_SELECTOR, '#video-title > yt-formatted-string')
    views_list_html = driver.find_elements(By.CSS_SELECTOR, '#metadata-line > span:nth-child(3)')

print("수집된 총 제목 개수 :" , len(titles_list_html))
print("수집된 총 조회수 개수 :" , len(views_list_html))

title_list, view_list = [] , []

for i in tqdm(range(50)) :
    title_list.append(titles_list_html[i].text)
    view_list.append(views_list_html[i].text)

print("영상 제목 수 :", len(title_list))
print("영상 조회 수 :", len(view_list))

# 웹 페이지에서 스크롤을 내려서 HTML요소의 개수를 정확히 맞추는 것은 사실 불가능하기 때문에
# 우리는 원하는 개수보다 조금 더 수집해서 원하는 개수 만큼만 따로 저장해서 사용하면 됨!

In [7]:
youtube_df = pd.DataFrame({'제목':title_list, '조회수':view_list}, index=range(1, len(title_list)+1))
youtube_df.index.name = 'No'
youtube_df

,제목,조회수
No,,
1,2025년 7월 10일 (목) 오늘의 주요 뉴스 #Shorts,조회수 2.6만회
2,"민주당 ""체포동의안 오면 다 가결 시킬 것""‥ #Shorts (MBC뉴스)",조회수 2.6만회
3,"[풀영상] 뉴스12 : 윤, 넉 달 만에 재구속…내란 재판 불출석 – 2025년 7...",조회수 3.2천회
4,[속보] 재구속 윤 전대통령 '내란재판' 불출석 - [MBC 뉴스속보] MBC뉴스 ...,6.1천명 시청 중
5,"[속보] 무죄 확정 박정훈 대령, 해병대 수사단장 직무 복귀 - [MBC 뉴스속보]...",965명 시청 중
6,"윤 전 대통령 구속영장 발부 - ""증거 인멸 염려"" - [LIVE] MBC 뉴스특보...",조회수 44만회
7,[오늘 이 뉴스] 이종섭·이시원 집 들이닥쳤다..국방부에도 특검팀 '우르르' (20...,조회수 7.3만회
8,[LIVE][KBS 뉴스특보] 윤석열 전 대통령 재구속/2025년 7월 10일(목)...,조회수 1.2만회
9,"넉 달 만에 재구속‥""외환 혐의 등 수사 탄력"" - [LIVE] MBC 뉴스투데이 ...",조회수 29만회


In [9]:
# 조회수에 '시청 중' 이라는 문자가 있는 행 지우기!
# str.contains("문자열") : 특정 문자의 포함 여부를 확인하는 함수, 시청중이 포함된 행이지 아닌지를 Trye, False로 알려줌
# ~youtube_df['조회수'] : 조회수라는 열을 꺼냄.

youtube_df[~youtube_df['조회수'].str.contains("시청 중")]     #조회수 컬럼에서 '시청 중' 문자열이 없는 행에 접근하는 조건

,제목,조회수
No,,
1,2025년 7월 10일 (목) 오늘의 주요 뉴스 #Shorts,조회수 2.6만회
2,"민주당 ""체포동의안 오면 다 가결 시킬 것""‥ #Shorts (MBC뉴스)",조회수 2.6만회
3,"[풀영상] 뉴스12 : 윤, 넉 달 만에 재구속…내란 재판 불출석 – 2025년 7...",조회수 3.2천회
6,"윤 전 대통령 구속영장 발부 - ""증거 인멸 염려"" - [LIVE] MBC 뉴스특보...",조회수 44만회
7,[오늘 이 뉴스] 이종섭·이시원 집 들이닥쳤다..국방부에도 특검팀 '우르르' (20...,조회수 7.3만회
8,[LIVE][KBS 뉴스특보] 윤석열 전 대통령 재구속/2025년 7월 10일(목)...,조회수 1.2만회
9,"넉 달 만에 재구속‥""외환 혐의 등 수사 탄력"" - [LIVE] MBC 뉴스투데이 ...",조회수 29만회
10,[풀영상] 뉴스광장 : 윤석열 전 대통령 재구속…“증거인멸 염려” - 2025년 7...,조회수 2.9만회
11,"윤, 124일 만에 재구속‥ ""증거인멸 우려"" - [LIVE] MBC 12시 뉴스 ...",조회수 4.1만회


In [10]:
# '조회수' 열에 '시청 중'이 포함된 행을 제거하고, 그 결과를 다시 youtube_df에 덮어씌움 (즉, 업데이트)

youtube_df = youtube_df[~youtube_df['조회수'].str.contains("시청 중")] 
youtube_df.shape

# youtube_df.shape : 필터링 후 남은 youtube_df의 행(row)과 열(column) 수를 확인하는 것, 튜플 (행 수, 열 수) 형식으로 나와

(48, 2)

In [11]:
youtube_df.index = range(1, youtube_df.shape[0]+1)  # outube_df.shape[0] → 행(row)의 개수, range(1, 행+1) → 예: range(1, 48)이면 1 ~ 47까지
youtube_df.index.name = 'No'
youtube_df

,제목,조회수
No,,
1,2025년 7월 10일 (목) 오늘의 주요 뉴스 #Shorts,조회수 2.6만회
2,"민주당 ""체포동의안 오면 다 가결 시킬 것""‥ #Shorts (MBC뉴스)",조회수 2.6만회
3,"[풀영상] 뉴스12 : 윤, 넉 달 만에 재구속…내란 재판 불출석 – 2025년 7...",조회수 3.2천회
4,"윤 전 대통령 구속영장 발부 - ""증거 인멸 염려"" - [LIVE] MBC 뉴스특보...",조회수 44만회
5,[오늘 이 뉴스] 이종섭·이시원 집 들이닥쳤다..국방부에도 특검팀 '우르르' (20...,조회수 7.3만회
6,[LIVE][KBS 뉴스특보] 윤석열 전 대통령 재구속/2025년 7월 10일(목)...,조회수 1.2만회
7,"넉 달 만에 재구속‥""외환 혐의 등 수사 탄력"" - [LIVE] MBC 뉴스투데이 ...",조회수 29만회
8,[풀영상] 뉴스광장 : 윤석열 전 대통령 재구속…“증거인멸 염려” - 2025년 7...,조회수 2.9만회
9,"윤, 124일 만에 재구속‥ ""증거인멸 우려"" - [LIVE] MBC 12시 뉴스 ...",조회수 4.1만회
